# Добавляем report во все классы

In [1]:
import joblib
from typing import Any, Dict, List, Tuple, Type, Union
import numpy as np
from matplotlib import pyplot as plt
from sklearn.base import BaseEstimator
from sklearn.utils import all_estimators
from sklearn.metrics import ConfusionMatrixDisplay, roc_curve
from sklearn.neighbors import NearestNeighbors
from Levenshtein import distance as lev_dist
import pandas as pd
# Импорт метрик для оценки модели (scores)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, r2_score,
    d2_absolute_error_score, ndcg_score, rand_score, dcg_score, fbeta_score,
    adjusted_rand_score, silhouette_score, completeness_score, homogeneity_score,
    jaccard_score, v_measure_score, brier_score_loss, d2_tweedie_score,
    cohen_kappa_score, d2_pinball_score, mutual_info_score, adjusted_mutual_info_score,
    average_precision_score, label_ranking_average_precision_score, balanced_accuracy_score,
    top_k_accuracy_score, calinski_harabasz_score, roc_auc_score
)
# Импорт метрик ошибок (errors)
from sklearn.metrics import (
    max_error, mean_absolute_percentage_error, median_absolute_error,
    mean_squared_log_error, mean_squared_error, mean_absolute_error
)
import sklearn

scores = (
    accuracy_score, precision_score, recall_score, f1_score, r2_score,
    d2_absolute_error_score, ndcg_score, rand_score, dcg_score, fbeta_score,
    adjusted_rand_score, silhouette_score, completeness_score, homogeneity_score,
    jaccard_score, v_measure_score, brier_score_loss, d2_tweedie_score,
    cohen_kappa_score, d2_pinball_score, mutual_info_score, adjusted_mutual_info_score,
    average_precision_score, label_ranking_average_precision_score, balanced_accuracy_score,
    top_k_accuracy_score, calinski_harabasz_score, roc_auc_score
)

errors = (
    max_error, mean_absolute_percentage_error, median_absolute_error,
    mean_squared_log_error, mean_squared_error, mean_absolute_error
)

In [3]:
class Model:
    def __init__(self, model: Type[BaseEstimator] = None):
        """
        Initialize the Model instance.

        Parameters:
        model (Type[BaseEstimator], optional): The model to use. Defaults to None.
        """
        all_models = self.check_model_type()

        if model:
            assert isinstance(model, tuple(all_models)), ('Incorrect input model type. '
                                                          f'Should be one of {type(self)} models from sklearn')
        self.__model: BaseEstimator = model  # Приватизируем атрибут model

    @property
    def model(self):
        return self.__model

    def check_model_type(self) -> List[Type[BaseEstimator]]:
        self.__model_types_with_names: List[Tuple[str, Type[BaseEstimator]]] = all_estimators(
            type_filter=type(self).__name__.lower())
        all_models = [t[1] for t in self.__model_types_with_names]
        return all_models

    def fit(self, X: Any, y: Any = None, *args: Any, **kwargs: Any) -> None:
        """
        Fit the model to the data.

        Parameters:
        X (Any): Training data.
        y (Any, optional): Target values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.
        """
        assert self.__model is not None, "Model is not defined."
        self.__model.fit(X, y, *args, **kwargs)

    def predict(self, X: Any, *args: Any, **kwargs: Any) -> Any:
        """
        Predict using the model.

        Parameters:
        X (Any): Data to predict.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        Any: Predicted values.
        """
        assert self.__model is not None, "Model is not defined."
        return self.__model.predict(X, *args, **kwargs)

    def predict_proba(self, X: Any, *args: Any, **kwargs: Any) -> Any:
        """
        Predict class probabilities using the model.

        Parameters:
        X (Any): Data to predict.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        Any: Predicted class probabilities.
        """
        classifier_models = [t[1] for t in all_estimators(type_filter='classifier')]
        assert isinstance(self.__model, tuple(classifier_models)), ('Incorrect model type for predict_proba. '
                                                                    f'Should be one of {classifier_models}')
        return self.__model.predict_proba(X, *args, **kwargs)

    def save_model(self, path: str, *args: Any, **kwargs: Any) -> None:
        """
        Save the model to a file.

        Parameters:
        path (str): The path to save the model.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.
        """
        assert self.__model is not None, "Model is not defined."
        joblib.dump(self, path, *args, **kwargs)  # Сохраняем текущий объект Model

    @classmethod
    def load_model(cls, path: str, *args: Any, **kwargs: Any) -> 'Model':
        """
        Load a model from a file.

        Parameters:
        path (str): The path to load the model from.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        Model: An instance of the Model class with the loaded model.
        """
        try:
            model_instance = joblib.load(path, *args, **kwargs)
            assert isinstance(model_instance, cls), "Loaded object is not an instance of the expected class."
            return model_instance
        except:
            raise ValueError("You're tying to load incorrect model")

    def fit_all(self, X: Any, y: Any = None, *args: Any, **kwargs: Any) -> Tuple[
        Dict[str, 'Model'], Dict[str, Exception]]:
        """
        Fit all available models to the data.

        Parameters:
        X (Any): Training data.
        y (Any, optional): Target values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        Tuple[Dict[str, Model], Dict[str, Exception]]: A tuple containing a dictionary of fitted models and a dictionary of errors.
        """
        fitted_models: Dict[str, 'Model'] = {}
        error_fitting: Dict[str, Exception] = {}

        for model_name, model_type in self.__model_types_with_names:
            try:
                model_instance = model_type()
                model_instance.fit(X, y, *args, **kwargs)
                wrapped_model = self.__class__(model_instance)
                fitted_models[model_name] = wrapped_model
            except Exception as e:
                error_fitting[model_name] = e

        return fitted_models, error_fitting

    def get_params(self, *args: Any, **kwargs: Any) -> Dict[str, Any]:
        """
        Get parameters of the model.

        Parameters:
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        Dict[str, Any]: Model parameters.
        """
        assert self.__model is not None, "Model is not defined."
        return self.__model.get_params(*args, **kwargs)

In [4]:
class Regressor(Model):
    def __init__(self, model: Type[BaseEstimator] = None) -> None:
        """
        Initialize the Regressor instance.

        Parameters:
        model (Type[BaseEstimator], optional): The model to use. Defaults to None.
        """
        super().__init__(model)

    def r2_score(self, y_test: Any, y_pred: Any, *args: Any, **kwargs: Any) -> float:
        """
        Calculate the R2 score.

        Parameters:
        y_test (Any): True values.
        y_pred (Any): Predicted values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        float: R2 score.
        """
        return r2_score(y_test, y_pred, *args, **kwargs)

    def mean_absolute_error(self, y_true: Any, y_pred: Any, *args: Any, **kwargs: Any) -> float:
        """
        Calculate the Mean Absolute Error (MAE).

        Parameters:
        y_true (Any): True values.
        y_pred (Any): Predicted values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        float: Mean Absolute Error.
        """
        return mean_absolute_error(y_true, y_pred, *args, **kwargs)

    def mean_squared_error(self, y_true: Any, y_pred: Any, *args: Any, **kwargs: Any) -> float:
        """
        Calculate the Mean Squared Error (MSE).

        Parameters:
        y_true (Any): True values.
        y_pred (Any): Predicted values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        float: Mean Squared Error.
        """
        return mean_squared_error(y_true, y_pred, *args, **kwargs)

    def root_mean_squared_error(self, y_true: Any, y_pred: Any, *args: Any, **kwargs: Any) -> float:
        """
        Calculate the Root Mean Squared Error (RMSE).

        Parameters:
        y_true (Any): True values.
        y_pred (Any): Predicted values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        float: Root Mean Squared Error.
        """
        return np.sqrt(mean_squared_error(y_true, y_pred, *args, **kwargs))

    def max_error(self, y_true: Any, y_pred: Any, *args: Any, **kwargs: Any) -> float:
        """
        Calculate the Max Error.

        Parameters:
        y_true (Any): True values.
        y_pred (Any): Predicted values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        float: Max Error.
        """
        return max_error(y_true, y_pred, *args, **kwargs)

    def mean_absolute_percentage_error(self, y_true: Any, y_pred: Any, *args: Any, **kwargs: Any) -> float:
        """
        Calculate the Mean Absolute Percentage Error (MAPE).

        Parameters:
        y_true (Any): True values.
        y_pred (Any): Predicted values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        float: Mean Absolute Percentage Error.
        """
        return mean_absolute_percentage_error(y_true, y_pred, *args, **kwargs)

    def median_absolute_error(self, y_true: Any, y_pred: Any, *args: Any, **kwargs: Any) -> float:
        """
        Calculate the Median Absolute Error (MedAE).

        Parameters:
        y_true (Any): True values.
        y_pred (Any): Predicted values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        float: Median Absolute Error.
        """
        return median_absolute_error(y_true, y_pred, *args, **kwargs)

    def mean_squared_log_error(self, y_true: Any, y_pred: Any, *args: Any, **kwargs: Any) -> float:
        """
        Calculate the Mean Squared Logarithmic Error (MSLE).

        Parameters:
        y_true (Any): True values.
        y_pred (Any): Predicted values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        float: Mean Squared Logarithmic Error.
        """
        return mean_squared_log_error(y_true, y_pred, *args, **kwargs)

    def d2_absolute_error_score(self, y_true: Any, y_pred: Any, *args: Any, **kwargs: Any) -> float:
        """
        Calculate the D2 Absolute Error Score.

        Parameters:
        y_true (Any): True values.
        y_pred (Any): Predicted values.
        *args (Any): Variable length argument list.
        **kwargs (Any): Arbitrary keyword arguments.

        Returns:
        float: D2 Absolute Error Score.
        """
        return d2_absolute_error_score(y_true, y_pred, *args, **kwargs)

    def root_mean_squared_log_error(self, y_true: Any, y_pred: Any, *args: Any, **kwargs: Any) -> float:
        """
        Calculate the Root Mean Squared Logarithmic Error (RMSLE).

        Parameters:
        y_true (array-like): True values.
        y_pred (array-like): Predicted values.

        Returns:
        float: Root Mean Squared Logarithmic Error.
        """
        return np.sqrt(mean_squared_log_error(y_true, y_pred, *args, **kwargs))

In [5]:
reg = Regressor()
reg.__dict__

{'_Model__model_types_with_names': [('ARDRegression',
   sklearn.linear_model._bayes.ARDRegression),
  ('AdaBoostRegressor', sklearn.ensemble._weight_boosting.AdaBoostRegressor),
  ('BaggingRegressor', sklearn.ensemble._bagging.BaggingRegressor),
  ('BayesianRidge', sklearn.linear_model._bayes.BayesianRidge),
  ('CCA', sklearn.cross_decomposition._pls.CCA),
  ('DecisionTreeRegressor', sklearn.tree._classes.DecisionTreeRegressor),
  ('DummyRegressor', sklearn.dummy.DummyRegressor),
  ('ElasticNet', sklearn.linear_model._coordinate_descent.ElasticNet),
  ('ElasticNetCV', sklearn.linear_model._coordinate_descent.ElasticNetCV),
  ('ExtraTreeRegressor', sklearn.tree._classes.ExtraTreeRegressor),
  ('ExtraTreesRegressor', sklearn.ensemble._forest.ExtraTreesRegressor),
  ('GammaRegressor', sklearn.linear_model._glm.glm.GammaRegressor),
  ('GaussianProcessRegressor',
   sklearn.gaussian_process._gpr.GaussianProcessRegressor),
  ('GradientBoostingRegressor',
   sklearn.ensemble._gb.GradientBoos